In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2024-08-03 13:28:33.893177: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-03 13:28:33.976814: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-03 13:28:34.071946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 13:28:34.157605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 13:28:34.177772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-03 13:28:34.290065: I tensorflow/core/platform/cpu_feature_gu

In [3]:
# Carregar os dados
data = pd.read_csv("/home/darkcover/Documentos/Out/dados/odds_200k.csv")
data = data.drop(columns=['Unnamed: 0'])
data = data.rename(columns={'Odd_Categoria': 'odd_saida'})

# Preprocessamento dos dados
# Converte os valores da coluna 'odd_saida' em binário (0 ou 1)
data['binaria'] = data['odd_saida'].apply(lambda x: 1 if x >= 5 else 0)

# Dividir os dados em arrays de 320 e 80 entradas
def split_data(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

# Definir as janelas de entrada e saída
window_size = 320
output_size = 80


In [6]:
# Preparar as entradas e saídas
X, y = split_data(data['binaria'].values, window_size, output_size)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Remodelar os dados para o formato adequado para o LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(window_size, 1)))
model.add(LSTM(50))
model.add(Dense(output_size, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Fazer previsões
predictions = model.predict(X_test)

# Verificar a precisão das previsões
accuracy = np.mean(np.round(predictions) == y_test)
print(f'Precisão: {accuracy:.2f}')

TypeError: split_data() takes 2 positional arguments but 3 were given

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Carregar os dados
data = pd.read_csv("/home/darkcover/Documentos/Out/dados/odds_200k.csv")
data = data.drop(columns=['Unnamed: 0'])
data = data.rename(columns={'Odd_Categoria': 'odd_saida'})

# Preprocessamento dos dados
data['binaria'] = data['odd_saida'].apply(lambda x: 1 if x >= 5 else 0)

# Função para dividir os dados em segmentos
def split_data(data, window_size, output_size):
    X, y = [], []
    for i in range(len(data) - window_size - output_size + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size:i+window_size+output_size])
    return np.array(X), np.array(y)

# Definir as janelas de entrada e saída
window_size = 320
output_size = 80

# Preparar as entradas e saídas
X, y = split_data(data['binaria'].values, window_size, output_size)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Remodelar os dados para o formato adequado para o LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(window_size, 1)))
model.add(LSTM(50))
model.add(Dense(output_size, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Fazer previsões
predictions = model.predict(X_test)

# Verificar a precisão das previsões
accuracy = np.mean(np.round(predictions) == y_test)
print(f'Precisão: {accuracy:.2f}')


/home/darkcover/Documentos/Out/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2024-08-03 13:33:03.783774: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 408780800 exceeds 10% of free system memory.
2024-08-03 13:33:04.551306: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102195200 exceeds 10% of free system memory.


Epoch 1/10
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.0011 - loss: 0.6283

2024-08-03 13:46:27.663345: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102195200 exceeds 10% of free system memory.
2024-08-03 13:46:28.583394: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25548800 exceeds 10% of free system memory.


2495/2495 ━━━━━━━━━━━━━━━━━━━━ 947s 378ms/step - accuracy: 0.0011 - loss: 0.6283 - val_accuracy: 0.0000e+00 - val_loss: 0.6260
Epoch 2/10
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 970s 389ms/step - accuracy: 0.0151 - loss: 0.6259 - val_accuracy: 0.0000e+00 - val_loss: 0.6260
Epoch 3/10
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 849s 340ms/step - accuracy: 0.0043 - loss: 0.6258 - val_accuracy: 0.0000e+00 - val_loss: 0.6260
Epoch 4/10
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 827s 331ms/step - accuracy: 0.0059 - loss: 0.6256 - val_accuracy: 0.0000e+00 - val_loss: 0.6260
Epoch 5/10
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 937s 375ms/step - accuracy: 0.0027 - loss: 0.6257 - val_accuracy: 0.0000e+00 - val_loss: 0.6259
Epoch 6/10
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 935s 375ms/step - accuracy: 0.0060 - loss: 0.6258 - val_accuracy: 0.0000e+00 - val_loss: 0.6260
Epoch 7/10
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 979s 374ms/step - accuracy: 0.0234 - loss: 0.6257 - val_accuracy: 0.0000e+00 - val_loss: 0.6259
Epoch 8/10
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 976